In [ ]:
import cv2
import os

# Define output folder path
output_folder = r"C:\Users\yuval\OneDrive\Documents\motion_output"

# Ensure the folder exists
os.makedirs(output_folder, exist_ok=True)

# Define output file path
output_file = os.path.join(output_folder, "motion_output.mp4")

# Open video file
cap = cv2.VideoCapture(r"C:\Users\yuval\Downloads\6824077-uhd_2160_3840_30fps.mp4")

# Set output video size (matching resized frames)
frame_width = 1080
frame_height = 1080
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Read the first frame
ret, prev_frame = cap.read()
prev_frame = cv2.resize(prev_frame, (frame_width, frame_height))  # Resize to match processing size
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to match previous frame size
    frame = cv2.resize(frame, (frame_width, frame_height))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute absolute difference
    diff = cv2.absdiff(prev_gray, gray)

    # Apply threshold to detect motion
    _, motion_mask = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)

    # Convert grayscale mask to 3-channel image for saving
    motion_color = cv2.cvtColor(motion_mask, cv2.COLOR_GRAY2BGR)

    # Write to output video
    out.write(motion_color)

    # Show results
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Motion Mask', motion_mask)

    # Update previous frame
    prev_gray = gray.copy()

    # Press 'q' to exit
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Motion detection output saved at: {output_file}")


In [1]:
import cv2
import numpy as np
import os


alpha = 0.01  
output_folder = r"C:\Users\yuval\OneDrive\Documents\motion_output"
os.makedirs(output_folder, exist_ok=True)


fps = 20.0  
frame_size = (640, 480) 

cap = cv2.VideoCapture(r"C:\Users\yuval\Downloads\6824077-uhd_2160_3840_30fps.mp4")  


bg_model = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    frame = cv2.resize(frame, frame_size)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    
    if bg_model is None:
        bg_model = gray.astype("float")
        continue

    
    cv2.accumulateWeighted(gray, bg_model, alpha)

   
    bg_diff = cv2.absdiff(gray, cv2.convertScaleAbs(bg_model))

    
    _, mask = cv2.threshold(bg_diff, 25, 255, cv2.THRESH_BINARY)


    cv2.imshow("Frame", frame)
    cv2.imshow("Background", cv2.convertScaleAbs(bg_model))
    cv2.imshow("Foreground Mask", mask)

  
    key = cv2.waitKey(0) & 0xFF  

    
    if key == ord('q'):
        print("Exiting...")
        break


cap.release()
cv2.destroyAllWindows()

Exiting...


In [2]:
import cv2
import numpy as np
import os


threshold = 0.5 
output_folder = r"C:\Users\yuval\OneDrive\Documents\scene"
os.makedirs(output_folder, exist_ok=True)


cap = cv2.VideoCapture(r"C:\Users\yuval\Downloads\scene.mp4")
frame_count = 0
scene_count = 0
prev_frame = None

def calculate_histogram_difference(frame1, frame2):
    """Calculate histogram difference between two frames."""
    hist1 = cv2.calcHist([frame1], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([frame2], [0], None, [256], [0, 256])
    
    # Normalize histograms
    cv2.normalize(hist1, hist1, 0, 1, cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, 0, 1, cv2.NORM_MINMAX)
    
    # Compare histograms using correlation
    diff = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    
    return diff

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break
    
    frame_count += 1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prev_frame is not None:
        
        diff = calculate_histogram_difference(prev_frame, gray)

        
        if diff < threshold:
            scene_count += 1
            print(f"Scene Change Detected at frame {frame_count} (Difference: {diff:.2f})")
            
            
            cv2.imwrite(os.path.join(output_folder, f"scene_{scene_count}.png"), frame)

       
        cv2.putText(frame, f"Diff: {diff:.2f}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    prev_frame = gray

   
    cv2.imshow("Frame", frame)
    
   
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

print(f"Total Scenes Detected: {scene_count}")

Scene Change Detected at frame 147 (Difference: 0.36)
Scene Change Detected at frame 198 (Difference: -0.02)
Scene Change Detected at frame 263 (Difference: 0.40)
Scene Change Detected at frame 463 (Difference: 0.35)
Scene Change Detected at frame 555 (Difference: 0.19)
End of video or error reading frame.
Total Scenes Detected: 5
